<a href="https://colab.research.google.com/github/anitakumar/ml/blob/main/ags_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

Breaks data into small blocks to answer the questions

In [ ]:
!pip install langchain langchain_community openai tiktoken

In [ ]:
!pip install llama-index

In [ ]:
# prompt: install these llama-index-huggingface, llama-index-embeddings-huggingface, transformers ,accelerate

!pip install llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface transformers accelerate


In [ ]:
!pip install llama-index-llms-huggingface

In [ ]:

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Document, ServiceContext, VectorStoreIndex, Settings
from llama_index.core.node_parser import SimpleNodeParser # each row is getting indexed. Each row will creare a document
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd


In [ ]:
df = pd.read_csv('HCP_Targeting_and_Segmentation_sample.csv')

In [ ]:
df.head()

,HCP_ID,Name,Specialty,Region,Prescribing_Trend,Research_Involvement,Engagement_Score
0,HCP00001,HCP_Name_1,Pediatrics,North,Low,Yes,Medium
1,HCP00002,HCP_Name_2,Pediatrics,North,Low,Yes,Low
2,HCP00003,HCP_Name_3,Cardiology,Central,Low,No,Medium
3,HCP00004,HCP_Name_4,Pediatrics,East,Medium,Yes,Low
4,HCP00005,HCP_Name_5,General Practice,Central,High,Yes,High


In [ ]:
#llm_model="Falconsai/medical_summarization" # gives error not using

In [ ]:
llm_model = "tiiuae/falcon-rw-1b"

In [ ]:
embedding_model="BAAI/bge-small-en"

In [ ]:
# llm_model = "tiiuae/falcon-rw-1b"
embedding_model = "BAAI/bge-small-en"

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(llm_model)

tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(llm_model) #loads model for casual language learning

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
# from transformers import AutoModelForSeq2SeqLM

# # ... other code ...

# model = AutoModelForSeq2SeqLM.from_pretrained(llm_model)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.falcon.configuration_falcon.FalconConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, Qwen2AudioConfig, SeamlessM4TConfig, SeamlessM4Tv2Config, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.

wrap into llama index

In [ ]:
llm = HuggingFaceLLM(model=model,
               tokenizer=tokenizer,
               context_window=1024,
               max_new_tokens=256,
               generate_kwargs={"temperature":0.7, "do_sample":True},
               )

In [ ]:
embed_model=HuggingFaceEmbedding(model_name=embedding_model)

In [ ]:
Settings.llm= llm
Settings.embed_model=embed_model

In [ ]:
df.columns

Index(['HCP_ID', 'Name', 'Specialty', 'Region', 'Prescribing_Trend',
       'Research_Involvement', 'Engagement_Score'],
      dtype='object')

In [ ]:
df.iterrows()

<generator object DataFrame.iterrows at 0x799223863560>

In [ ]:
documents= [   Document(
        text=f"HCP_ID: {row['HCP_ID']}, Name: {row['Name']}, Specialty: {row['Specialty']}, "
             f"Region: {row['Region']}, Prescribing_Trend: {row['Prescribing_Trend']}, "
             f"Research_Involvement: {row['Research_Involvement']}, Engagement_Score: {row['Engagement_Score']}",
        metadata={"row_index": idx}  # Add metadata for reference (optional)
    )
for idx, row in df.iterrows()]

In [ ]:
df.shape

(99, 7)

In [ ]:
len(documents)

99

so all of each 99 documents will be chunked separetely. A small example belwo of overlaps etcs

In [ ]:
text ="""It is used for question-answering with sources an LlamaIndex graph data structure.from langchain_community"""
doc= Document(text=text)
p1=SimpleNodeParser.from_defaults(chunk_size=5, chunk_overlap=2)
n1=p1.get_nodes_from_documents([doc])
for i in n1:
  print(i.get_text())

Metadata length (0) is close to chunk size (5). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
It is used for
used for question-answering
with sources an
sources an LlamaIndex
graph data structure.
structure.from langchain_community


In [ ]:
parser= SimpleNodeParser()

In [ ]:
nodes=parser.get_nodes_from_documents(documents)

In [ ]:
nodes

Building a query. Now store in vector store index

In [ ]:
index=VectorStoreIndex(nodes=nodes)
retriver=index.as_retriever()

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
df.head()

,HCP_ID,Name,Specialty,Region,Prescribing_Trend,Research_Involvement,Engagement_Score
0,HCP00001,HCP_Name_1,Pediatrics,North,Low,Yes,Medium
1,HCP00002,HCP_Name_2,Pediatrics,North,Low,Yes,Low
2,HCP00003,HCP_Name_3,Cardiology,Central,Low,No,Medium
3,HCP00004,HCP_Name_4,Pediatrics,East,Medium,Yes,Low
4,HCP00005,HCP_Name_5,General Practice,Central,High,Yes,High


In [ ]:
query ="What do the medical specilaities in the East Region"

Below is the query form the retriever

In [ ]:
n2=retriver.retrieve(query)

In [ ]:
for i in n2:
  print(i.get_text())

HCP_ID: HCP00049, Name: HCP_Name_49, Specialty: Dermatology, Region: East, Prescribing_Trend: Low, Research_Involvement: Yes, Engagement_Score: Low
HCP_ID: HCP00022, Name: HCP_Name_22, Specialty: Dermatology, Region: East, Prescribing_Trend: Medium, Research_Involvement: Yes, Engagement_Score: High


#only go 2 records . as it was form the retriever

In [ ]:
response=query_engine.query(query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(response)


---------------------
What is the current level of engagement of the HCPs in the East Region?
Answer: "Low"
---------------------
The clinical relevance of this question is at the level of: "Medium"
---------------------
What is the current level of engagement of the HCPs in the East Region?
Answer: "High"
---------------------
Given the context information and not prior knowledge, answer the query.
Query: What do the medical specilaities in the East Region
Answer: "Medium"
---------------------
What is the current level of engagement of the HCPs in the East Region?
Answer: "Medium"
---------------------
Given the context information and not prior knowledge, answer the query.
Query: What do the medical specilaities in the East Region
Answer: "High"
---------------------
What is the current level of engagement of the HCPs in the East Region?
Answer: "High"
---------------------
Given the context information and not prior knowledge, answer the query.
Query: What do the medical specilait